# ==========================================
# 1. ライブラリのインポートと設定
# ==========================================

In [ ]:
# 🔧 環境セットアップ (Google Drive マウント版)
# =================================================
import os
import sys
import torch

# 1. Google Driveをマウント
print("🔄 Google Driveをマウント中...")
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Driveマウント完了")
except Exception as e:
    print(f"❌ Google Driveマウントエラー: {e}")
    raise

# 2. プロジェクトのパスを設定
# ※Google Drive内のプロジェクトフォルダパスを指定してください
PROJECT_PATH = '/content/drive/MyDrive/sae_pj2'

if not os.path.exists(PROJECT_PATH):
    print(f"❌ プロジェクトフォルダが見つかりません: {PROJECT_PATH}")
    print("💡 PROJECT_PATHの変数が正しいか、フォルダが同期されているか確認してください。")
else:
    print(f"✅ プロジェクトフォルダ発見: {PROJECT_PATH}")

    # 3. Pythonがプロジェクト内の.pyファイルを読み込めるようにパスを追加
    if PROJECT_PATH not in sys.path:
        sys.path.insert(0, PROJECT_PATH)
        print(f"   🐍 Pythonパスに追加しました")

    # colabフォルダもパスに追加
    colab_path = os.path.join(PROJECT_PATH, 'colab')
    if colab_path not in sys.path:
        sys.path.insert(0, colab_path)
        print(f"   🐍 colabパスに追加しました")

    # 4. 作業ディレクトリをプロジェクトフォルダに移動
    os.chdir(PROJECT_PATH)
    print(f"   📁 作業ディレクトリを変更しました: {os.getcwd()}")

    # 4.5 必須ファイルの存在チェック
    required_items = [
        ('config.py', '設定ファイル'),
        ('colab/feedback_analyzer.py', 'メインロジック'),
        ('eval_dataset', 'データセットフォルダ'),
        ('eval_dataset/feedback.jsonl', 'Feedbackデータ')
    ]
    for path, label in required_items:
        exists = os.path.exists(path)
        icon = '✅' if exists else '⚠️'
        kind = '📄' if os.path.isfile(path) else '📁'
        print(f"   {icon} 必須確認: {kind} {path} - {label}{'' if exists else '（見つからない場合は同期を確認）'}")

# 5. 依存関係のインストール
print("\n📦 依存関係をインストール中...")
req_file = 'requirements-colab.txt'
if os.path.exists(req_file):
    try:
        get_ipython().system(f"pip install -q -r {req_file}")
        print("✅ 依存関係インストール完了 (requirements-colab.txtを使用)")
    except Exception as e:
        print(f"⚠️ 依存関係インストールで警告/エラー: {e}")
        print("   必要に応じて個別に !pip install <package> を実行してください")
else:
    print(f"⚠️ {req_file} が見つかりません。必要なライブラリが不足する可能性があります。")

# 6. 環境確認
print("\n🚀 環境セットアップ完了！")
if torch.cuda.is_available():
    try:
        print(f"   GPU検出: {torch.cuda.get_device_name(0)}")
        memory_total = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"   GPU Memory: {memory_total:.1f}GB")
    except Exception:
        print("   GPU検出: 利用可能 (デバイス名の取得に失敗)")
else:
    print("   ❌ GPU利用不可 (CPUモード)")


In [ ]:
# メモリ最適化設定
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
torch.set_grad_enabled(True) # 勾配計算を有効化

In [ ]:
from huggingface_hub import login
login()  # 画面の指示に従いトークン入力

# ==========================================
# 2. データ処理ロジック (指示書に基づく実装)
# ==========================================

In [ ]:
def yield_sycophancy_samples(data: Dict[str, Any]) -> Generator[Dict[str, Any], None, None]:
    """
    JSONデータからAttribution Patching用のサンプルを生成するジェネレータ

    Logic:
    1. 各質問について、まず 'base' テンプレートの回答を探す。
    2. 同じ質問内で sycophancy_flag=1 の回答を探す。
    3. ペアが見つかれば yield する。
    """
    results = data.get("results", [])

    for result in results:
        variations = result.get("variations", [])
        question_id = result.get("question_id")

        # 1. Base回答の特定
        base_variation = None
        for var in variations:
            t_type = var.get("template_type")
            # template_typeが 'base' または None/空文字 の場合をBaseとみなす
            if t_type == "base" or t_type == "(base)" or not t_type:
                base_variation = var
                break

        # Baseが存在しない場合はスキップ
        if not base_variation:
            continue

        # 2. ターゲット（迎合）回答の特定とペアリング
        for target_variation in variations:
            # 自分自身（Base）はスキップ
            if target_variation is base_variation:
                continue

            # sycophancy_flag が 1 (迎合している) もののみ対象
            if target_variation.get("sycophancy_flag") == 1:
                yield {
                    "question_id": question_id,
                    "template_type": target_variation.get("template_type"),
                    "prompt": target_variation.get("prompt"),
                    "response": target_variation.get("response"),       # Target Token用
                    "base_response": base_variation.get("response")     # Base Token (Contrastive) 用
                }

# ==========================================
# 3. Attribution Patching 分析クラス
# ==========================================

In [ ]:
class AttributionPatchingAnalyzer:
    def __init__(self, model: HookedTransformer, sae: SAE, config: Any):
        self.model = model
        self.sae = sae
        self.config = config
        self.hook_name = sae.cfg.hook_name

    def _find_answer_start_position(self, full_tokens: torch.Tensor, prompt_str: str) -> int:
        """
        プロンプトの終わり（回答の始まり）のトークン位置を特定する
        """
        # プロンプト単体でのトークン長を取得
        # Note: BOSトークン等の扱いに注意。Gemmaはadd_bos_token=Trueがデフォルト
        prompt_tokens = self.model.to_tokens(prompt_str, prepend_bos=True)
        return prompt_tokens.shape[1] - 1  # 0-indexed なので -1 (最後のトークン位置)

    def calculate_atp_for_sample(self, sample: Dict[str, Any]) -> Dict[str, Any]:
        """
        1サンプルに対するAttribution Patchingを実行
        """
        prompt = sample["prompt"]
        response = sample["response"]
        base_response = sample["base_response"]

        # 1. トークン化 (Teacher Forcing Input)
        # プロンプト + 実際の回答
        full_text = prompt + response
        input_tokens = self.model.to_tokens(full_text, prepend_bos=True)

        # 回答開始位置（プロンプトの最後のトークン位置）を特定
        # ここが「次のトークン（回答の1文字目）」を予測する位置になる
        target_pos = self._find_answer_start_position(input_tokens, prompt)

        # 入力長チェック (モデルのコンテキスト長を超えないか)
        if input_tokens.shape[1] > self.model.cfg.n_ctx:
            return {"error": "Sequence too long"}

        # 2. Target Token と Base Token の ID を取得
        # 実際に生成された回答の1トークン目
        target_token_id = input_tokens[0, target_pos + 1].item()

        # Base回答の1トークン目 (Contrastive Pairs)
        # Base回答をトークン化して最初のトークンを取得
        # Prompt + Base Response でトークン化しないと、結合部のトークンが変わる可能性があるため注意
        base_full_text = prompt + base_response
        base_input_tokens = self.model.to_tokens(base_full_text, prepend_bos=True)

        # プロンプト長が同じであると仮定（同じプロンプトなので）
        # ただしトークナイザの挙動によりズレる可能性もゼロではないため、安全策をとるならここでBase側のposも再計算推奨
        # 今回は簡略化のため、Promptが同一なら長さも同一とみなす
        try:
            base_token_id = base_input_tokens[0, target_pos + 1].item()
        except IndexError:
            # Base回答が空、あるいは極端に短い場合
            return {"error": "Base response too short"}

        # TargetとBaseが同じトークンなら、Logit Diffは0になり勾配も0なのでスキップ可能
        if target_token_id == base_token_id:
            return {"skipped": "Target and Base tokens are identical"}

        # 3. Forward Pass & Metric Calculation
        self.model.eval()
        self.model.zero_grad()

        # フック内でデータをキャプチャするためのコンテナ
        feature_acts_storage = {}

        def atp_hook(activation, hook):
            """
            Activationを取得し、SAEを通して勾配を流すフック
            """
            # activation: [batch, seq, d_model]
            # ターゲット位置（回答直前）のみを抽出
            # batch=1 前提
            target_act = activation[:, target_pos:target_pos+1, :]

            # SAE Encode (Feature Activation計算)
            # SAEの入力次元に合わせて調整
            f_acts = self.sae.encode(target_act) # [1, 1, n_features]

            # 勾配計算のために保存 (retain_grad重要)
            f_acts.requires_grad_(True)
            f_acts.retain_grad()
            feature_acts_storage['acts'] = f_acts

            # SAE Decode (Reconstruction)
            x_hat = self.sae.decode(f_acts)

            # Gradient Trick:
            # Forward: 元のActivation (x) をそのまま流す (Teacher Forcingの精度維持)
            # Backward: Reconstruction (x_hat) を通して勾配を流す (SAE特徴量へのPathを作る)
            # x_out = x_hat + (x - x_hat).detach()
            # これにより、Metricの勾配は x_hat -> f_acts と伝播する

            x_out = x_hat + (target_act - x_hat).detach()

            # 元のシーケンスに戻す
            activation[:, target_pos:target_pos+1, :] = x_out
            return activation

        # モデル実行
        try:
            logits = self.model.run_with_hooks(
                input_tokens,
                fwd_hooks=[(self.hook_name, atp_hook)]
            )

            # 4. Metric Calculation (Logit Difference)
            # target_posの位置での予測を見る
            target_logit = logits[0, target_pos, target_token_id]
            base_logit = logits[0, target_pos, base_token_id]
            metric = target_logit - base_logit

            # 5. Backward Pass
            metric.backward()

            # 6. AtP Score Calculation
            # Score = Activation * Gradient
            f_acts = feature_acts_storage['acts']
            f_grad = f_acts.grad

            if f_acts is None or f_grad is None:
                return {"error": "Failed to capture gradients"}

            atp_scores = (f_acts * f_grad).detach().cpu().squeeze() # [n_features]

            # 結果の抽出（Top-K & Non-zero）
            # メモリ節約のため、スコアが高いものだけを保存
            top_k = 50
            top_indices = torch.topk(atp_scores.abs(), k=top_k).indices

            result_scores = {}
            for idx in top_indices:
                idx_val = idx.item()
                score = atp_scores[idx_val].item()
                activation_val = f_acts[0, 0, idx_val].item()

                # 活性化していない（0または非常に小さい）特徴は因果効果がないため除外してもよいが
                # 負の勾配（抑制すべきだったのに発火した等）も重要なので、AtPスコア絶対値で保存
                result_scores[str(idx_val)] = {
                    "atp_score": score,
                    "activation": activation_val,
                    "gradient": f_grad[0, 0, idx_val].item()
                }

            return {
                "status": "success",
                "target_token": self.model.to_string(target_token_id),
                "base_token": self.model.to_string(base_token_id),
                "logit_diff": metric.item(),
                "atp_scores": result_scores
            }

        except RuntimeError as e:
            if "out of memory" in str(e).lower():
                torch.cuda.empty_cache()
                return {"error": "OOM"}
            raise e
        finally:
            # メモリクリーンアップ
            self.model.zero_grad()
            del feature_acts_storage
            if 'logits' in locals(): del logits
            if 'f_acts' in locals(): del f_acts
            if 'f_grad' in locals(): del f_grad
            torch.cuda.empty_cache()

# ==========================================
# 4. メイン実行スクリプト
# ==========================================

In [ ]:
def run_attribution_patching_pipeline():
    # パス設定
    PROJECT_PATH = '/content/drive/MyDrive/sae_pj2'
    INPUT_JSON_PATH = os.path.join(PROJECT_PATH, "results/feedback/feedback_analysis_gemma-2-9b-it_20251117_0-100_labeled.json") # ※ファイル名は適宜変更してください
    OUTPUT_JSON_PATH = os.path.join(PROJECT_PATH, "results/feedback/atp_results_gemma-2-9b-it.json")

    # ファイル存在確認
    if not os.path.exists(INPUT_JSON_PATH):
        # テスト用に最新の結果ファイルを探す
        search_dir = Path(os.path.join(PROJECT_PATH, "results/feedback"))
        files = list(search_dir.glob("feedback_analysis_*.json"))
        if files:
            INPUT_JSON_PATH = str(sorted(files)[-1])
            print(f"⚠️ 指定されたファイルが見つからないため、最新のファイルを使用します: {INPUT_JSON_PATH}")
        else:
            raise FileNotFoundError("Input JSON file not found.")

    # 1. データの読み込み
    print(f"📂 Loading data from {INPUT_JSON_PATH}...")
    with open(INPUT_JSON_PATH, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # 2. モデルとSAEの準備
    # Note: 既にメモリにロードされている場合は再利用可能ですが、ここでは安全のためconfigからロードを想定
    from config import FEEDBACK_GEMMA2_9B_IT_CONFIG

    # メモリ不足対策: batch_size=1, gradient checkpointingなどは無効化（推論モードなので）
    print("🔄 Loading Model & SAE...")
    # 既存のanalyzerがあればそれを使う、なければ新規作成
    # ここでは簡易的に直書きします
    device = "cuda" if torch.cuda.is_available() else "cpu"

    model = HookedTransformer.from_pretrained_no_processing(
        FEEDBACK_GEMMA2_9B_IT_CONFIG.model.name,
        device=device,
        dtype=torch.bfloat16
    )

    sae, _, _ = SAE.from_pretrained(
        release=FEEDBACK_GEMMA2_9B_IT_CONFIG.model.sae_release,
        sae_id=FEEDBACK_GEMMA2_9B_IT_CONFIG.model.sae_id,
        device=device
    )

    analyzer = AttributionPatchingAnalyzer(model, sae, FEEDBACK_GEMMA2_9B_IT_CONFIG)

    # 3. メインループ
    results = []
    samples_gen = yield_sycophancy_samples(data)

    # ジェネレータをリスト化してtqdmで回す（メモリ注意）
    # 大量データの場合はバッチ処理推奨
    samples = list(samples_gen)
    print(f"🚀 Starting ATP analysis for {len(samples)} samples...")

    for i, sample in enumerate(tqdm(samples)):
        res = analyzer.calculate_atp_for_sample(sample)

        # 結果をサンプル情報に結合
        sample_result = {
            "question_id": sample["question_id"],
            "template_type": sample["template_type"],
            "atp_analysis": res
        }
        results.append(sample_result)

        # 定期的に保存（クラッシュ対策）
        if (i + 1) % 10 == 0:
            with open(OUTPUT_JSON_PATH, 'w', encoding='utf-8') as f:
                json.dump({"results": results}, f, indent=2, ensure_ascii=False)
            # ガベージコレクション
            gc.collect()
            torch.cuda.empty_cache()

    # 4. 最終保存
    with open(OUTPUT_JSON_PATH, 'w', encoding='utf-8') as f:
        json.dump({"results": results}, f, indent=2, ensure_ascii=False)

    print(f"✅ Analysis completed. Saved to {OUTPUT_JSON_PATH}")

# ==========================================
# 4. 実行
# ==========================================


In [ ]:
# 実行
if __name__ == "__main__":
    # Colab環境での実行を想定
    try:
        run_attribution_patching_pipeline()
    except Exception as e:
        print(f"❌ Error during execution: {e}")
        import traceback
        traceback.print_exc()